In [ ]:
import pandas as pd
import geopandas as gpd
import datetime
import numpy as np
from shapely.geometry import Point, LineString
import shapely.wkt
import os
import json


In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def downloadData(textfile, X = True, Y = True, Z = True, output = 'csv', outputFile = 'data.csv'):
    '''
    This function takes a text file from OSC in the phone
    The axis we want to consider to compute the final vector (X, Y, Z)
    And the output formats and file
    and returns a dataframe with the V values for each gps coordinate point
    '''
    #read original data from file within track.txt.gz used by OSC to store sensor data
    data = pd.read_csv(textfile,sep=';',
                   skiprows=[0],
                   skipfooter=1,
                   usecols=[0,1,2,9,10,11,15],
                   header=None,
                   engine = 'python')

    #naming of columns 
    data.columns = ['timestamp','long','lat','accelerationX','accelerationY','accelerationZ','photoIndex'] 
    
    #conversion into timestamp
    #dates = []
    #for i in range(data.shape[0]):
    #    try:
    #        dates.append(datetime.datetime.fromtimestamp(data['timestamp'].iloc[i]))
    #    except :
    #        print 'Error with row:', i
    #data['timestamp'] = dates        
    
    #remove all empty rows except timestamp
    emtpy = data.iloc[:,1:].isnull().sum(axis=1) == data.shape[1]-1
    data = data.loc[~emtpy,:]
    data.index=range(data.shape[0])
    

    data.dropna(axis=0,how='all',
                subset = ['long','lat','accelerationX','accelerationY','accelerationZ','photoIndex'],
                inplace=True)
    
    
    #CREATE A PHOTOS DATAFRAME
    dataPhotos = data.loc[~data.photoIndex.isnull(),['timestamp','photoIndex']]
    data.drop(['photoIndex'],axis=1,inplace=True)
    

    #'''
    #create the geography
    gpsDataPoints =  data.loc[~ (data['long'].isnull()),['timestamp','long','lat']]
    gpsDataPoints['pointIndex'] = gpsDataPoints.index
    
    geometry = []
    for i in range(len(gpsDataPoints.index)):
        if i == (len(gpsDataPoints.index)-1):
            line = np.nan
        else:
            #get start and end points for each line
            startPoint = Point(gpsDataPoints['long'].loc[gpsDataPoints.index[i]], gpsDataPoints['lat'].loc[gpsDataPoints.index[i]])
            endPoint = Point(gpsDataPoints['long'].loc[gpsDataPoints.index[i+1]], gpsDataPoints['lat'].loc[gpsDataPoints.index[i+1]])
            #convert to shapely wkt
            line = LineString([startPoint,endPoint]).wkt
            geometry.append(shapely.wkt.loads(line).centroid)
    
    gpsDataPoints = gpsDataPoints.iloc[:-1]
    crs = {'init': 'epsg:4326'}
    gpsDataPoints = gpd.GeoDataFrame(gpsDataPoints, crs=crs, geometry=geometry)
    
    #Asign to data the index of the points with GPS data
    data.drop(['timestamp'],axis=1,inplace=True)
    data = data.merge(gpsDataPoints.drop(['timestamp','geometry'],axis=1),how='left')
    data['pointIndex'] = data['pointIndex'].fillna(method='ffill')
    
    #shift data lag 1 to take the vector of the difference in axis XYZ
    dataShifted = data.shift(1)
    dataShifted.drop(['long','lat','pointIndex'],axis=1,inplace=True)
    dataShifted.columns = ['accelerationXShift','accelerationYShift','accelerationZShift']
    
    #concatenate datasets
    data = pd.concat([data,dataShifted],axis=1)
    data.drop(['long','lat'],axis=1,inplace=True)
    data.dropna(axis=0,how='any',inplace=True)
    
    #compute vector
    
    data['V'] = np.sqrt((data.accelerationX-data.accelerationXShift) ** 2 * X + \
    (data.accelerationY-data.accelerationYShift) ** 2 * Y + \
    (data.accelerationZ-data.accelerationZShift) ** 2 * Z) 
    
    #get the sum of every lag BY line defined by the starting point (with GPS data)
    vectorInformation = data.loc[:,['pointIndex','V']].groupby(by=['pointIndex']).sum()
    vectorInformation.reset_index(inplace=True)
    gpsDataPoints = gpsDataPoints.merge(vectorInformation)
    
    #gpsDataPoints.timestamp = gpsDataPoints.timestamp.map(lambda x: str(x))
    
    photoPoints = []
    photoV = []

    for i in range(dataPhotos.shape[0]):
        if i == (dataPhotos.shape[0]-1):
            break
        else:
            dataAggregated = gpsDataPoints.loc[(gpsDataPoints.timestamp > dataPhotos.timestamp.iloc[i]) & (gpsDataPoints.timestamp < dataPhotos.timestamp.iloc[i+1]),:]
            photoV.append(dataAggregated.V.sum())
            photoPoints.append(dataAggregated.geometry.iloc[0])
    dataPhotos = dataPhotos.iloc[:dataPhotos.shape[0]-1,:]        
    dataPhotos['V'] = photoV
    dataPhotos['geometry'] = photoPoints
    crs = {'init': 'epsg:4326'}
    dataPhotos = gpd.GeoDataFrame(dataPhotos, crs=crs, geometry=dataPhotos.geometry)
    
    if output == 'csv':
        gpsDataPoints.to_csv(outputFile)
    elif output == 'shp':
        gpsDataPoints.to_file(outputFile)
    else:
        raise NameError('You can only export to csv or shp files')
    
    #return gpsDataPoints
    #'''
    return (gpsDataPoints,dataPhotos)

In [ ]:
X = True
Y = True
Z = True
output = 'csv'
outputFile = 'data.csv'
dataType = 'accelerometer'
OSCid=464547
diccion = loadOSCdata.getOSCjson(OSCid)
diccion = diccion['osv']

In [ ]:
url='http://openstreetcam.org/'+diccion['meta_data_filename']
url          

In [ ]:
#read the accelerometer data with loadOSCdata function
#tupla = downloadData('/home/pipe/Dropbox/NYU/classes/Capstone/OSCstats/track.txt', X = X, Y = Y, Z = Z, output = output, outputFile = outputFile)


In [ ]:
tupla = downloadData(url, X = X, Y = Y, Z = Z, output = output, outputFile = outputFile)


In [ ]:
data = tupla[0]
dataPhotos = tupla[1]

In [ ]:
data.head()

In [ ]:
dataPhotos.head()

In [ ]:
diccion['photos'][0]

In [ ]:
diccion['photos'][1]

In [ ]:
fotos = loadOSCdata.queryOSCapi(OSCid = 464547, output = 'shp', outputFile = 'photos',dataType='photos')
fotos.head()

In [ ]:
curl 'https://southcentralus.api.cognitive.microsoft.com/customvision/v1.0/Prediction/16d2699a-3afd-4548-a7d3-2263abf5be63/url?iterationId=d6c29a8a-8001-49eb-b497-9f8b7109dfe7&application=quicktest' -H 'Origin: https://customvision.ai' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8,es;q=0.6' -H 'Prediction-Key: a6886e6f5b834133880fcde81d496cdd' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36' -H 'Content-Type: application/json;charset=UTF-8' -H 'Training-Key: 4fb7652c838747da9989682b2d382150' -H 'Accept: application/json, text/plain, */*' -H 'Referer: https://customvision.ai/projects/16d2699a-3afd-4548-a7d3-2263abf5be63' -H 'Connection: keep-alive' -H 'DNT: 1' --data-binary '{"Url":"http://storage4.openstreetcam.org/files/photo/2017/6/3/proc/441358_ff07a_5932edaa9492a.jpg"}' --compressed